<a href="https://colab.research.google.com/github/shreyac435/math-question-classifier/blob/main/math_question_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from IPython.display import display
display("Resetting notebook metadata")



'Resetting notebook metadata'

In [2]:
import os
import zipfile
import json

os.listdir('/content')
zipname = 'dataset.zip'
zipfile_obj = zipfile.ZipFile('/content/' + zipname, 'r') #unzipped the file here
zipfile_obj.extractall('/content/data')
zipfile_obj.close()
os.listdir('/content/data')
os.listdir('/content/data/MATH')
os.listdir('/content/data/MATH/train')


['algebra',
 'prealgebra',
 'number_theory',
 'counting_and_probability',
 'geometry',
 'precalculus',
 'intermediate_algebra']

In [3]:
questions = []
topics = []
basepath = '/content/data/MATH/train'

for folder in os.listdir(basepath):
    folderpath = basepath + '/' + folder
    for file in os.listdir(folderpath):
        filepath = folderpath + '/' + file
        f = open(filepath, 'r')
        data = json.load(f)
        f.close()
        questions.append(data['problem'])
        topics.append(data['type'])
print(len(questions))
print(questions[0])
print(topics[0])


7500
What is the minimum value of $z$ if $z=x^2+2y^2+6x-4y+22?$
Algebra


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
qtrain, qtest, ttrain, ttest = train_test_split(
    questions, topics, test_size=0.2, random_state=20)
vectorizer = TfidfVectorizer(
    lowercase=True,
    stop_words='english',
    ngram_range=(1, 2))
qtrain_vec = vectorizer.fit_transform(qtrain)
qtest_vec = vectorizer.transform(qtest)

In [5]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
nb_model = MultinomialNB()
nb_model.fit(qtrain_vec, ttrain)
nb_pred = nb_model.predict(qtest_vec)
nb_acc = accuracy_score(ttest, nb_pred)
print("Naive Bayes Accuracy:", nb_acc)

Naive Bayes Accuracy: 0.6786666666666666


In [6]:
from sklearn.svm import LinearSVC
svm_model = LinearSVC()
svm_model.fit(qtrain_vec, ttrain)
svm_pred = svm_model.predict(qtest_vec)
svm_acc = accuracy_score(ttest, svm_pred)
print("Linear SVM Accuracy:", svm_acc)

Linear SVM Accuracy: 0.734


In [7]:
for i in range(5):
    print("ACTUAL TOPIC:")
    print(ttest[i])
    print("NB PREDICTED:")
    print(nb_pred[i])
    print("SVM PREDICTED:")
    print(svm_pred[i])
    print("\n")

ACTUAL TOPIC:
Algebra
NB PREDICTED:
Algebra
SVM PREDICTED:
Algebra


ACTUAL TOPIC:
Counting & Probability
NB PREDICTED:
Number Theory
SVM PREDICTED:
Number Theory


ACTUAL TOPIC:
Precalculus
NB PREDICTED:
Precalculus
SVM PREDICTED:
Precalculus


ACTUAL TOPIC:
Number Theory
NB PREDICTED:
Number Theory
SVM PREDICTED:
Number Theory


ACTUAL TOPIC:
Algebra
NB PREDICTED:
Algebra
SVM PREDICTED:
Algebra




In [8]:
sample_qs = []
for folder in os.listdir(basepath):
    folderpath = basepath + '/' + folder
    for file in os.listdir(folderpath):
        filepath = folderpath + '/' + file
        f = open(filepath, 'r')
        data = json.load(f)
        f.close()
        sample_qs.append(data['problem'])
        if len(sample_qs) == 10:
            break
    if len(sample_qs) == 10:
        break
sample_qs[:3]

['What is the minimum value of $z$ if $z=x^2+2y^2+6x-4y+22?$',
 'If $a*b=a^2+ab-b^2$, find $3*2$.',
 'If a 3\'\' by 3\'\' square is added at each successive stage, what will be the area of the rectangle at Stage 6, in square inches?\n\n[asy]size(250); real textsize = 10pt;\ndraw(unitsquare);\ndraw(shift(1.5*right)*unitsquare);\ndraw(shift(2.5*right)*unitsquare);\ndraw(shift(4*right)*unitsquare);\ndraw(shift(5*right)*unitsquare);\ndraw(shift(6*right)*unitsquare);\nlabel("Stage 1",(.5,0),S,fontsize(textsize));\nlabel("Stage 2",(2.5,0),S,fontsize(textsize));\nlabel("Stage 3",(5.5,0),S,fontsize(textsize));[/asy]']

In [9]:
!pip install -q transformers accelerate sentencepiece

In [11]:

import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "google/flan-t5-xl"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16
)

def explain_question(q):
    prompt = "Explain step by step in detail for a high school student:\n" + q
    tokens = tokenizer(prompt, return_tensors="pt").to(model.device)
    output = model.generate(**tokens, max_new_tokens=200)
    return tokenizer.decode(output[0], skip_special_tokens=True)



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.45G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [12]:
for q in sample_qs[:2]:
    print("QUESTION:")
    print(q)
    print("\nLLM EXPLANATION:")
    print(explain_question(q))

QUESTION:
What is the minimum value of $z$ if $z=x^2+2y^2+6x-4y+22?$

LLM EXPLANATION:
The minimum value of $z$ is 0 because z is the product of x and y. The answer: 0 because z is the product of x and y.
QUESTION:
If $a*b=a^2+ab-b^2$, find $3*2$.

LLM EXPLANATION:
Let a be the number of times a squared. Let b be the number of times b squared. Plugging these into the equation, we get a + b = a * b. Combining like terms, we get a + b = a * b. Dividing both sides by a, we get b = a * b - b * a. Multiplying both sides by a, we get b = a * b - a * b. Then multiplying both sides by 2, we get 3 * 2 = 3 * 2 = 3 * 2 = 3 * 2 = 3 * 2 = 3 * 2 = 3 * 2 = 3 * 2 = 3 * 2 = 3 * 2 = 3 * 2 = 3 * 2 = 3 * 2 = 3 * 2 = 3 * 2 = 3 * 2 = 3 * 2 = 3


In [16]:
while True:
    user_q = input("\nEnter a math question (or type exit): ")
    if user_q.lower() == "exit":
        break
    user_vec = vectorizer.transform([user_q])
    predicted_topic = svm_model.predict(user_vec)[0]
    print("Predicted Topic:", predicted_topic)
    print("\nExplanation:\n")
    print(explain_question(user_q))

#just a demo, isn't properly working, some questions to ask
#solve for x, x+3=0
#is 3 a prime number?
#

KeyboardInterrupt: Interrupted by user

In [ ]:
!pip install -q wandb

import wandb

wandb.login()

wandb.init(project="math-question-classifier")

wandb.log({
    "Naive Bayes Accuracy": nb_acc,
    "Linear SVM Accuracy": svm_acc,
    "Features": "TF-IDF (unigram + bigram)"
})

wandb.finish()
